In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [7]:
fake= pd.read_csv('fake.csv')
true= pd.read_csv('true.csv')

In [8]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [9]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [10]:
fake['class'] = 0
true['class'] = 1

In [11]:
data=pd.concat([fake, true], axis=0)

In [12]:
data.sample(10)

,title,text,subject,date,class
16934,Kenyan police say four people killed during op...,NAIROBI (Reuters) - Kenya s police said on Fri...,worldnews,"October 20, 2017",1
13924,Rights groups say outside monitors needed for ...,YANGON (Reuters) - Human rights groups called ...,worldnews,"November 24, 2017",1
5549,Man Falsely ID’d As Dallas Shooter Says Cops ...,"Whether you realize it or not, the Second Amen...",News,"July 8, 2016",0
1339,White House Officials Push For Kushner To Ste...,Donald Trump keeps digging himself deeper and ...,News,"May 28, 2017",0
5604,Obama Singing ‘Happy Birthday’ To Malia Is Th...,"On July 4, we celebrate our independence each ...",News,"July 5, 2016",0
16101,France releases brother of ex-Burkina presiden...,PARIS (Reuters) - A court in Paris on Monday r...,worldnews,"October 30, 2017",1
10256,¿Qué bolá? Obama jokes with Cuban humorist in ...,HAVANA (Reuters) - U.S. President Barack Obama...,politicsNews,"March 20, 2016",1
3049,Sam Bee On #GoldenShowerGate: ‘It’s Number On...,Trump has had a rough week! We d feel sorry fo...,News,"January 12, 2017",0
10844,"Senate passes 'provocative' China bill, Cruz l...",WASHINGTON (Reuters) - The U.S. Senate on Frid...,politicsNews,"February 12, 2016",1
859,"Factbox: What's in tax bill from Trump, House ...",(Reuters) - Republicans in the U.S. House of R...,politicsNews,"November 2, 2017",1


In [15]:
data=data.drop(["title", "subject", "date"], axis=1)

In [16]:
data.reset_index(inplace=True)

In [17]:
data.drop(['index'],axis=1, inplace=True)

In [18]:
data.sample(5)

,text,class
21773,Meet Alice Miller who single-handedly scr*wed ...,0
29859,WASHINGTON (Reuters) - President-elect Donald ...,1
5824,This is one of the most epic smack downs of a ...,0
29812,WASHINGTON (Reuters) - Urged on by U.S. Presid...,1
38040,HARARE (Reuters) - Zimbabwe President Robert M...,1


In [19]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\[.*?\]', '', text)
    text = re.sub(r'\\W', ' ', text)
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    text = re.sub(r'<.*?>+', '', text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)
    text = re.sub(r'\n', '', text)
    text = re.sub(r"\w*\d\w*", '', text)
    return text

In [20]:
data["text"] = data["text"].apply(clean_text)

In [21]:
x=data["text"]
y=data["class"] 
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [22]:
vectorizer=TfidfVectorizer()
xtrain=vectorizer.fit_transform(xtrain)
xv_test=vectorizer.transform(xtest)

In [23]:
lr=LogisticRegression()
lr.fit(xtrain, ytrain)

LogisticRegression()

In [24]:
predictions=lr.predict(xv_test)
lr.score(xv_test, ytest)

0.985924276169265

In [25]:
print(classification_report(ytest, predictions))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225

